In [1]:
# put absolute path here
DATA_DIR = r"C:\Users\User\Downloads\saos-dump-23.02.2018.tar\saos-dump-23.02.2018\data\json"

In [2]:
# making usage of my helper .py file in jupyter environment on any computer
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from file_utils import *
data_manager = DataManager(DATA_DIR)

In [3]:
# Actual homework starts here
import re
generator = data_manager.judgments_generator(year=2009)

In [4]:
def contain_digit(x):
    return re.search(r'\d', x) is not None

def remove_html(x):
    return re.sub("<[^>]*>", "", x)

def remove_linebreaks(x):
    return re.sub("-\n", "", x)

In [5]:
from collections import defaultdict
bigrams = defaultdict(int)
frequency_list = defaultdict(int)
              
while True:
    try:
        content = generator.__next__()
        text = remove_linebreaks(remove_html(content["textContent"]))
        words_found = [x.lower() for x in re.findall(r'\b\w\w+\b', text, re.UNICODE) 
                       if not contain_digit(x)]
        for index in range(len(words_found)-1):
            word1 = words_found[index]
            word2 = words_found[index+1]
            bigrams[word1, word2] += 1
            frequency_list[words_found[index]] += 1
        frequency_list[words_found[len(words_found)-1]] += 1    
    except StopIteration:
        break

In [6]:
sorted(bigrams.items(), key=lambda x: x[1], reverse=True)[:30]   

[(('art', 'ust'), 63755),
 (('nr', 'poz'), 24794),
 (('na', 'podstawie'), 21634),
 (('ust', 'pkt'), 18758),
 (('ust', 'ustawy'), 16731),
 (('zamówień', 'publicznych'), 15098),
 (('dz', 'nr'), 13621),
 (('przez', 'zamawiającego'), 12406),
 (('podstawie', 'art'), 12140),
 (('ustawy', 'dnia'), 11538),
 (('sygn', 'akt'), 10518),
 (('na', 'rzecz'), 10195),
 (('się', 'do'), 10133),
 (('ustawy', 'pzp'), 10119),
 (('pkt', 'ustawy'), 9515),
 (('art', 'art'), 9479),
 (('sąd', 'najwyższy'), 9345),
 (('art', 'ustawy'), 9075),
 (('nie', 'jest'), 8825),
 (('co', 'do'), 8012),
 (('nie', 'może'), 7973),
 (('poz', 'ze'), 7959),
 (('ze', 'zm'), 7676),
 (('ust', 'konstytucji'), 7015),
 (('sądu', 'najwyższego'), 6928),
 (('dnia', 'stycznia'), 6911),
 (('sąd', 'okręgowy'), 6684),
 (('sądu', 'okręgowego'), 6614),
 (('udziału', 'postępowaniu'), 6596),
 (('art', 'pkt'), 6569)]

In [7]:
import math

words_num = sum(frequency_list.values())
bigrams_num = sum(bigrams.values())

print("Number of words is: %d" % words_num)
print("Number of bigrams is: %d" % bigrams_num)

def calculate_PMI(bigram):
    return math.log((
        (bigrams[bigram]/bigrams_num)/(
            (frequency_list[bigram[0]]/words_num) * (frequency_list[bigram[1]]/words_num)
        )
    ))

Number of words is: 8975183
Number of bigrams is: 8970646


In [8]:
pmis = {}
for bigram in bigrams.keys():
    pmis[bigram] = calculate_PMI(bigram)

sorted(pmis.items(), key=lambda x: x[1], reverse=True)[:30]     

[(('napawania', 'łukowego'), 16.010479514926725),
 (('przyjeżdżają', 'mochody'), 16.010479514926725),
 (('systematycznością', 'stabilnością'), 16.010479514926725),
 (('osadnika', 'śużla'), 16.010479514926725),
 (('puhb', 'cewogaz'), 16.010479514926725),
 (('uścikowiec', 'oborniki'), 16.010479514926725),
 (('zaspawaniu', 'oszlifowaniu'), 16.010479514926725),
 (('transmisjami', 'piłkarskimi'), 16.010479514926725),
 (('rozjaśnione', 'rozbielone'), 16.010479514926725),
 (('diagności', 'laboratoryjni'), 16.010479514926725),
 (('histochemicznych', 'immunopatologicznych'), 16.010479514926725),
 (('immunopatologicznych', 'mikroskopii'), 16.010479514926725),
 (('przydomowego', 'ogródka'), 16.010479514926725),
 (('przetwórcom', 'rybnym'), 16.010479514926725),
 (('książeczką', 'wkładową'), 16.010479514926725),
 (('wkładową', 'walutową'), 16.010479514926725),
 (('walutową', 'książeczka'), 16.010479514926725),
 (('kobylarnia', 'brzoza'), 16.010479514926725),
 (('societe', 'anonyme'), 16.01047951492

In [9]:
import numpy as np

def llr(k):
    return 2 * sum(k) * (H(k) - H([k[0] + k[1], k[2] + k[3]]) - H([k[0] + k[2], k[1] + k[3]]))

def table(bigram):
    
    bigrams_num = bigrams[(bigram[0], bigram[1])] + bigrams[(bigram[0], bigram[1])]
    
    return [
        bigrams[bigram],
        frequency_list[bigram[1]] - bigrams[bigram],
        frequency_list[bigram[0]] - bigrams[bigram],
        bigrams_num + bigrams[bigram] - frequency_list[bigram[1]] - frequency_list[bigram[0]]
    ]

def H(k):
    N = sum(k)
    return sum([el / N * math.log(el / N + int(el == 0)) for el in k])

logs = {}
for bigram in bigrams.keys():
    logs[bigram] = llr(table(bigram))

sorted(logs.items(), key=lambda x: x[1], reverse=True)[:30]  

[(('art', 'ust'), 506774.59219576477),
 (('nr', 'poz'), 270069.14650384255),
 (('zamówień', 'publicznych'), 204473.137278883),
 (('na', 'podstawie'), 167234.1288804641),
 (('dz', 'nr'), 146956.41287150426),
 (('ust', 'pkt'), 135966.0069371712),
 (('sygn', 'akt'), 135374.9478726683),
 (('sąd', 'najwyższy'), 103094.00035551893),
 (('ust', 'ustawy'), 102071.01117142508),
 (('trybunał', 'konstytucyjny'), 90235.07786853376),
 (('ze', 'zm'), 85265.55161486538),
 (('gr', 'słownie'), 83587.00344005921),
 (('sądu', 'najwyższego'), 77988.17972408858),
 (('ustawy', 'pzp'), 76381.7607765416),
 (('na', 'rzecz'), 76203.55075778105),
 (('przez', 'zamawiającego'), 75856.01663444626),
 (('sądu', 'okręgowego'), 74326.55875399089),
 (('udziału', 'postępowaniu'), 74232.11589492403),
 (('kio', 'uzp'), 74124.17136129543),
 (('sąd', 'okręgowy'), 72609.22686746098),
 (('podstawie', 'art'), 69340.97590788447),
 (('otk', 'zu'), 67095.48600554072),
 (('urzędu', 'zamówień'), 66738.62774770924),
 (('zero', 'groszy

In [11]:
print(bigrams['art', 'ust'])

63755
